<a href="https://colab.research.google.com/github/Akshay1897/Sarcasm-Detection/blob/master/Sarcasm_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
PATH_TO_DATA = '../input/sarcasm/train-balanced-sarcasm.csv'